# Параметрическое исследование экспоненциального роста

In [ ]:
using DrWatson
@quickactivate "project"
using DifferentialEquations
using DataFrames
using Plots
using JLD2
using BenchmarkTools

script_name = splitext(basename(PROGRAM_FILE))[1]
mkpath(plotsdir(script_name))
mkpath(datadir(script_name))

function exponential_growth!(du, u, p, t)
    α = p
    du[1] = α * u[1]
end

Базовый набор параметров

In [ ]:
base_params = Dict(
    :u0 => [1.0],
    :α => 0.3,
    :tspan => (0.0, 10.0),
    :solver => Tsit5(),
    :saveat => 0.1
)

println("Базовые параметры эксперимента:")
for (key, value) in base_params
    println("  $key = $value")
end

Функция для запуска одного эксперимента

In [ ]:
function run_single_experiment(params::Dict)
    u0 = params[:u0]
    α = params[:α]
    tspan = params[:tspan]
    solver = params[:solver]
    saveat = params[:saveat]

    prob = ODEProblem(exponential_growth!, u0, tspan, α)
    sol = solve(prob, solver; saveat=saveat)

    final_population = last(sol.u)[1]
    doubling_time = log(2) / α

    return Dict(
        "solution" => sol,
        "time_points" => sol.t,
        "population_values" => first.(sol.u),
        "final_population" => final_population,
        "doubling_time" => doubling_time,
        "parameters" => params
    )
end

Запуск базового эксперимента

In [ ]:
data, path = produce_or_load(
    datadir(script_name, "single"),
    base_params,
    run_single_experiment,
    prefix = "exp_growth",
    tag = false,
    verbose = true
)

println("\nРезультаты базового эксперимента:")
println("  Финальная популяция: ", data["final_population"])
println("  Время удвоения: ", round(data["doubling_time"]; digits=2))
println("  Файл результатов: ", path)

Визуализация базового эксперимента

In [ ]:
p1 = plot(data["time_points"], data["population_values"],
          label="α = $(base_params[:α])",
          xlabel="Время t", ylabel="Популяция u(t)",
          title="Экспоненциальный рост (базовый эксперимент)",
          lw=2, legend=:topleft, grid=true)

savefig(plotsdir(script_name, "single_experiment.png"))

Параметрическое сканирование

In [ ]:
param_grid = Dict(
    :u0 => [[1.0]],
    :α => [0.1, 0.3, 0.5, 0.8, 1.0],
    :tspan => [(0.0, 10.0)],
    :solver => [Tsit5()],
    :saveat => [0.1]
)

using DrWatson: dict_list
all_params = dict_list(param_grid)

println("\n" * "="^60)
println("ПАРАМЕТРИЧЕСКОЕ СКАНИРОВАНИЕ")
println("Всего комбинаций параметров: ", length(all_params))
println("Исследуемые значения α: ", param_grid[:α])
println("="^60)

Запуск всех экспериментов

In [ ]:
all_results = []
all_dfs = []

for (i, params) in enumerate(all_params)
    println("Прогресс: $i/$(length(all_params)) | α = $(params[:α])")

    data, path = produce_or_load(
        datadir(script_name, "parametric_scan"),
        params,
        run_single_experiment,
        prefix = "scan",
        tag = false,
        verbose = false
    )

    result_summary = merge(params, Dict(
        :final_population => data["final_population"],
        :doubling_time => data["doubling_time"],
        :filepath => path
    ))

    push!(all_results, result_summary)

    df = DataFrame(
        t = data["time_points"],
        u = data["population_values"],
        α = fill(params[:α], length(data["time_points"]))
    )
    push!(all_dfs, df)
end

Анализ результатов

In [ ]:
results_df = DataFrame(all_results)
println("\nСводная таблица результатов:")
println(results_df[:, [:α, :final_population, :doubling_time]])

Сравнительный график

In [ ]:
p2 = plot(size=(800, 500), dpi=150)

for params in all_params
    data, _ = produce_or_load(
        datadir(script_name, "parametric_scan"),
        params,
        run_single_experiment,
        prefix = "scan"
    )
    plot!(p2, data["time_points"], data["population_values"],
          label="α = $(params[:α])", lw=2, alpha=0.8)
end

plot!(p2, xlabel="Время t", ylabel="Популяция u(t)",
      title="Параметрическое исследование: влияние α на рост",
      legend=:topleft, grid=true)

savefig(plotsdir(script_name, "parametric_scan_comparison.png"))

График зависимости времени удвоения от α

In [ ]:
p3 = scatter(results_df.α, results_df.doubling_time,
             label="Численное решение",
             xlabel="Скорость роста α",
             ylabel="Время удвоения t₂",
             title="Зависимость времени удвоения от α",
             markersize=8, markercolor=:red, legend=:topright)

α_range = 0.1:0.01:1.0
plot!(p3, α_range, log.(2) ./ α_range,
      label="Теория: t₂ = ln(2)/α",
      lw=2, linestyle=:dash, linecolor=:blue)

savefig(plotsdir(script_name, "doubling_time_vs_alpha.png"))

Сохранение всех результатов

In [ ]:
@save datadir(script_name, "all_results.jld2") base_params param_grid all_params results_df
@save datadir(script_name, "all_plots.jld2") p1 p2 p3

println("\n" * "="^60)
println("ЛАБОРАТОРНАЯ РАБОТА ЗАВЕРШЕНА")
println("="^60)
println("\nРезультаты сохранены в:")
println("  - data/$(script_name)/single/ - базовый эксперимент")
println("  - data/$(script_name)/parametric_scan/ - параметрическое сканирование")
println("  - data/$(script_name)/all_results.jld2 - сводные данные")
println("  - plots/$(script_name)/ - все графики")